In [39]:
import numpy as np
import numpy.linalg as lin
import numpy.random as rnd

def fbSubs(LU, b):
    (rows, columns) = LU.shape

    # solve LZ = b
    z = np.full(rows, 0, dtype=np.number)
    for row in range(rows):
        sub = np.dot(LU[row], z)
        z[row] = b[row] - sub

    # solve RX = Z
    x = np.full(rows, 0, dtype=np.number)
    for row in reversed(range(rows)):
        sub = np.dot(LU[row], x)
        x[row] = (z[row] - sub)/LU[row, row]

    return x

def LU(A):
    (rows, columns) = A.shape
    idx = np.arange(rows)

    # decompose inplace into LU matrix
    for column in range(columns):
        max = column + np.argmax(np.abs(A[column:,column]))

        (A[column, :], A[max, :]) = (A[max, :].copy(), A[column, :].copy()) # use copy to be able to swap values
        (idx[column], idx[max]) = (idx[max].copy(), idx[column].copy())

        for row in range(column + 1, columns):
            scalar = A[row, column] / A[column, column]
            A[row, column:columns] -= (A[column] * scalar)[column:columns]
            A[row, column] = scalar

    return (A, idx)

def linsolve(A, b):
    (lu, idx) = LU(A)
    x = fbSubs(lu, b[idx])
    return x


# random orthogonal matrix
def rndOrtho(n):
    S = rnd.rand(n,n)
    S = S - S.T
    O = lin.solve(S - np.identity(n), S + np.identity(n))
    return O

# random matrix with specified condition number
def rndCond(n, cond):
    d = np.logspace(-np.log10(cond)/2, np.log10(cond)/2,n)
    A = np.diag(d)
    U,V = rndOrtho(n), rndOrtho(n)
    return U@A@V.T


In [40]:
# test lu with special matrix for gauss pivoting
A= np.array([[6., 3., 3.],
             [1., 3., 6.],
             [3., 6., 3.]])

(Lu, idx) = LU(A.copy())
L,R = np.tril(Lu,-1)+np.identity(3), np.triu(Lu)
assert( np.linalg.norm(L@R - A[idx]) < 1e-8)


n = 6 # size of test matrix

# test fbSubs
for k in range(1000):
    Lu = np.array( np.random.rand(n,n) )
    rhs = np.array(np.random.rand(n))
    x = fbSubs(Lu, rhs)

    L,R = np.tril(Lu,-1)+np.identity(n), np.triu(Lu)

    assert( np.linalg.norm(rhs - L@R@x) < 1e-8)

# test LU
for k in range(1000):
    A = np.array( np.random.rand(n,n) )
    (Lu, idx) = LU(A.copy())
    L,R = np.tril(Lu,-1)+np.identity(n), np.triu(Lu)
    assert( np.linalg.norm(L@R - A[idx]) < 1e-8)

# test linsolve
for k in range(1000):
    A = np.random.rand(n,n)
    rhs = np.random.rand(n)
    x = linsolve(A.copy(), rhs)
    assert( np.linalg.norm(rhs - A @ x) < 1e-8)

for k in range(1000):
    A = rndCond(n, 1e14)
    # code